# Создание скрипта для парсинга диалогов

Необходимо написать скрипт для парсинга диалогов из файла test_data.csv. 

Главные задачи, которые должен выполнять скрипт:
- Извлекать реплики с приветствием – где менеджер поздоровался. 
- Извлекать реплики, где менеджер представил себя. 
- Извлекать имя менеджера. 
- Извлекать название компании. 
- Извлекать реплики, где менеджер попрощался.
- Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»

План работы:
1. Загрузим и изучим данные
2. После изучения данных выберем способ поиска и извлечения необходимых нам сущностей.
3. Отчёт о проделанной работе и вывод.

In [1]:
# Импорт библиотек
import pandas as pd
import numpy as np

from yargy import Parser, rule
from yargy.pipelines import morph_pipeline
from yargy.predicates import type, normalized
from yargy.tokenizer import MorphTokenizer
from yargy import interpretation
from yargy.interpretation import fact

from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NamesExtractor,
    Doc
)

In [2]:
# pip install natasha

## Загрузка и исследование данных

In [3]:
# Загрузка данных
df = pd.read_csv('test_data.csv')

In [4]:
# Первый взгляд на данные
df.head()

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага


In [5]:
# Общая информация о данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   dlg_id  480 non-null    int64 
 1   line_n  480 non-null    int64 
 2   role    480 non-null    object
 3   text    480 non-null    object
dtypes: int64(2), object(2)
memory usage: 15.1+ KB


In [6]:
df.describe()

,dlg_id,line_n
count,480.000000,480.000000
mean,2.581250,48.056250
std,1.943262,35.355501
min,0.000000,0.000000
25%,1.000000,19.750000
50%,2.000000,40.500000
75%,5.000000,72.000000
max,5.000000,142.000000


In [7]:
# Посмотрим сколько диалогов, и сколько фраз в каждом диалоге
df['dlg_id'].value_counts()

5    143
0    109
2     85
1     55
3     53
4     35
Name: dlg_id, dtype: int64

In [8]:
# Посмотрим на любую реплику 
df.loc[319,'text']

'А вот а ну вот помните айдар вам не подходило то что вот вы ставили конечную дату платежа и у вас промежуточные платежи по ним задачи вот не выставляли то есть вы получали задачу на конечную дату вот и вот это было ну тем фактором который вам не подошел именно в этом виде этих платежей вот а'

In [9]:
# Проверим на дубликаты
df.duplicated().sum()

0

**Вывод**

1. Представленный набор данных содержит 480 строк с репликами в шести диалогах от 35 до 143 реплик в каждом. Для построения качественного парсера необходимой нам информации, данных маловао. Думаю, что на таком количестве данных возможно создать базовое решение, которое затем необходимо будет тестировать и дорабатывать на большем объёме данных.
2. Судя по первичному анализу текстов, информация получена из системы по переводу голоса в текст. В текстах отсутствуют знаки препинания, имена написаны не с загланой буквы. Это может негативно сказаться на качестве поиска необходимых нам сущностей.
3. Пропусков и дубликатов в данных нет.

## Создание парсера

Исходя из анализа имеемых данных, создание парсера будем производить следующим образом:
1. Извлекать реплики с приветствием и реплики, где менеджер попрощался - не составит большого труда. Для этого можно воспользоваться просто регулярными выражениями, но я решил применить Yargy парсер. Факт нахождения таких реплик будем заносить в дополнительнно созданные колонки `hello` и `goodbye`.
2. Извлекать имя менеджера будем с помошью Yargy парсера и библиотеки Natasha. Будем заносить факт представления менеджера в колонку `introduce`, а извлечённое имя в колонку `manager_name`.
3. Извлекать названия компаний было-бы правильно составив словарь компаний. Но так как датасет у нас не очень большой и словарь будет совсем маленький(2 компании), то извлекать названия компаний будем по ключевому слову 'компания' с помощью методов Yargy парсера и дополнительной логики. Имя компании занесём в колонку `company_name`.
4. Проверку требования к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом», будем осуществлять по информации из итоговой таблицы, которую будем составлять в процессе парсинга.
6. Для того, чтобы ускорить прицесс парсинга и не искать уже найденные в рамках диалога сущности, а также с целью более удобного отображения результатов, в процессе парсинга будем заполнять таблицу результатов с полями: 
    - `hello` - флаг менеджер поприветствовал клиента;
    - `introduce` - флаг менеджер представился;
    - `manager_name` - имя менеджера;
    - `goodbye` - флаг менеджер попращался;
    - `company_name` - имя компании.


In [10]:
# Создаём экземпляры
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
names_extractor = NamesExtractor(morph_vocab)

NAME = type('RU')

### Парсер приветствия и прощания

Создаём парсер по списку приветствия и прощания.

In [11]:
# Парсер приветствия
hello_pipline = morph_pipeline(['здравствуйте', 
                                'привет', 
                                'приветствую', 
                                'добрый день', 
                                'добрый вечер', 
                                'доброе утро', 
                                'доброй ночи'
                              ])
parser_hello = Parser(rule(hello_pipline))

In [12]:
# Парсер прощания
goodbye_pipline = morph_pipeline(['до свидания', 
                                  'до встречи', 
                                  'до завтра', 
                                  'пока'
                                ])
parser_goodbye = Parser(rule(goodbye_pipline))

### Парсер представления менеджера и извлечения имени

Просто применив извлечение имени средствами библиотеки Natasha, можно найти очень много лишнего, а имя `Максим` он не определяет, зато определяет `Добрый` как имя. Поэтому пришлось действовать так:
- Сначала ищем слова, указывающие на представление: `меня зовут`, `это`, `я`;
- Затем извлекаем слова перед, после и между этими словами;
- Проверяем имя это или нет.

In [13]:
# ПРЕДСТАВЛЕНИЕ МЕНЕДЖЕРА
# Создаём правило
Intro = fact('Name', ['before', 'between', 'after'])
before  = NAME.interpretation(Intro.before.custom(str))
between = NAME.interpretation(Intro.between.custom(str))
after   = NAME.interpretation(Intro.after.custom(str))

introduce = morph_pipeline(['меня', 'это', 'я'])

intro_rule = rule(before.optional(),
                  introduce,
                  between.optional(),
                  normalized('зовут').optional(), 
                  after.optional(),
).interpretation(Intro)
# Создаём парсер на основе правила
parser_intro = Parser(intro_rule)

# Здесь одним этим парсером не обойтись - необходима дополнительная логика - завернём её в функцию

def manager_name_extractor(text):
    """ Принимаем текст,
        ищем есть ли слова представления,
        Если находим - возвращаем флаг и имя менеджера
    """
    matches_intro = parser_intro.find(text)
    if matches_intro:
        # определяем имя менеджера в позициях по приоритету 
        if matches_intro.fact.between:
            manager_name = matches_intro.fact.between
        elif matches_intro.fact.after:
            manager_name = matches_intro.fact.after
        elif matches_intro.fact.before:
            manager_name = matches_intro.fact.before
        # Проверяем, то что мы нашли действительно имя?
        pobable_name = names_extractor.find(manager_name)
        if pobable_name:
            if pobable_name.fact.first:
                # Если нашли имя - возвращаем флаг и имя
                return True, manager_name
        # Если не нашли - возвращаем флаг - False и имя NaN
    return False, np.nan

### Парсер имени компании

Имя компании будем искать следующим образом:
- Ищем слово `компания`, `фирма` или `организация`;
- Извлекаем два слова после слова из первого пункта;
- Если слова два - ищем между ними синтаксическую связь. Если она есть - считаем что название компании состоит из двух слов. В имеемом датасете названия компаний состоят максимум из двух слов. Данный парсер необходимо будет доработать, если в датасете будут компании с названием из большего числа слов. 

In [14]:
# НАЗВАНИЕ КОМПАНИИ
# Создаём правило


Company = fact('company_name', ['first', 'second'])

first  = NAME.interpretation(Company.first.custom(str))
second = NAME.interpretation(Company.second.custom(str))

company = morph_pipeline(['компания', 'фирма', 'организация'])

company_rule = rule(company,
                    first.optional(),
                    second.optional()
).interpretation(Company)
# Создаём парсер на основе правила
parser_company = Parser(company_rule)


def company_name_extractor(text):
    """ Принимаем текст,
        ищем есть ли слова указывающие на название компании,
        Если находим - имя компании
    """
    company_name = np.nan
    matches_company = parser_company.find(text)
    if matches_company:
        # берём следующее слово после 'комания', считаем его названием компании
        company_name = matches_company.fact.first
        # Проверяем на синтаксическую связь со следующим словом
        doc_name = Doc(text)
        doc_name.segment(segmenter)
        doc_name.tag_morph(morph_tagger)
        doc_name.parse_syntax(syntax_parser)
        token_first = [token for token in doc_name.tokens if token.start == matches_company.tokens[1].span.start][0]
        token_second = [token for token in doc_name.tokens if token.start == matches_company.tokens[2].span.start][0]
        company_name = token_first.text
        if (token_first.head_id == token_second.id or
            token_first.id == token_second.head_id):
            # Если синтаксическая связь есть - добавляем это слово к первому слову названия
            company_name += ' ' + token_second.text
    return company_name

### Общий парсер

Собираем парсеры, созданные выше в один общий парсер.
Поиск необходимых сущностей будем осуществлять следующим образом:
- Создадим функцию поиска необходимых сущностей в строке набора данных;
- Проитерируем эту aункцию для каждой строки набора данных;
- Для исключения повторного поиска и удобного представления итогов парсинга создадим таблицу итогов.

In [15]:
def insite_parcer(line):
    """ Принимаем на вход строку из датафрейма,
        возвращаем:
           Флаги:
            - менеджер поздоровался;
            - менеджер представился;
            - менеджер попращался
           Данные:
            - имя менеджера;
            - название компании.
    """
    # Инициализация ответов
    hello = np.nan
    introduce = np.nan
    manager_name = np.nan
    goodbye = np.nan
    company_name = np.nan
    # Часто встречающиеся поля входящей строки выделим в отдельные переменные
    text = line['text']
    dlg_id = line['dlg_id']
    
    
    # Если в строке не менеджер - то нечего проверять - сразу на выход
    if line['role'] != 'manager':
        return hello, introduce, manager_name,  goodbye, company_name
    
    
    # Если менеджер - начинаем с работать со строкой
    # Если начинаем работу с новым диалогом - создаём запись о новом диалоге в таблице результатов
    if not (dlg_id in results.index):
        results.loc[dlg_id] = {'hello': False,
                               'introduce': False,
                               'goodbye': False,
                              }
    
    # ПРИВЕТСТВИЕ
    if not(results.loc[dlg_id,'hello']):
        # Если менеджер в данном диалоге ещё не здоровался  - ищем
        if  parser_hello.find(text):
            hello = True
            results.loc[dlg_id,'hello'] = True
    
    
    # ПРЕДСТАВЛЕНИЕ МЕНЕДЖЕРА
    if not(results.loc[dlg_id,'introduce']):
        # Если менеджер в данном диалоге ещё не представлялся  - ищем
        introduce, manager_name = manager_name_extractor(text)
        results.loc[dlg_id,'introduce'] = introduce
        results.loc[dlg_id,'manager_name'] = manager_name
               
    
    # НАЗВАНИЕ КОМПАНИИ
    if pd.isnull(results.loc[dlg_id,'company_name']):
        # Если менеджер в данном диалоге ещё не называл компанию - ищем
        company_name = company_name_extractor(text)
        results.loc[dlg_id,'company_name'] = company_name
            
            
    # ПРОЩАНИЕ
    if not(results.loc[dlg_id,'goodbye']):
        # Если менеджер в данном диалоге ещё не прощался - ищем
        if  parser_goodbye.find(text):
            goodbye = True
            results.loc[dlg_id,'goodbye'] = True
    
            
    return hello, introduce, manager_name,  goodbye, company_name

In [16]:
# Создадим таблицу с итогами парсинга. Ещё она поможет не искать уже найденные сущности
results = pd.DataFrame(columns=['hello', 'introduce', 'manager_name', 'goodbye', 'company_name'])


# Список колонок с итогами парсинга
parce_columns = ['hello', 'introduce', 'manager_name', 'goodbye', 'company_name']
# Запускаем парсер
df[parce_columns] = [_ for _ in df.apply(insite_parcer, axis=1)]


# Добавляем колонку, в которй будет логическое "и" колонок Hello и Goodbye
results['hello_and_goodbye'] = results['hello'] & results['goodbye']


# Отображение таблицы результатов
results

,hello,introduce,manager_name,goodbye,company_name,hello_and_goodbye
0,True,True,ангелина,True,диджитал бизнес,True
1,True,True,ангелина,True,диджитал бизнес,True
2,True,True,ангелина,False,диджитал бизнес,False
3,True,True,максим,False,китобизнес,False
4,False,False,NaN,True,NaN,False
5,False,True,анастасия,True,NaN,False


In [17]:
# Основной набор данных с не пустыми результатами поиска
df[df['hello'].notna() | 
   (df['introduce'].notna() & df['introduce']) | 
   df['manager_name'].notna() | 
   df['goodbye'].notna() | 
   df['company_name'].notna()]

,dlg_id,line_n,role,text,hello,introduce,manager_name,goodbye,company_name
1,0,1,manager,Алло здравствуйте,True,False,NaN,NaN,NaN
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,NaN,True,ангелина,NaN,диджитал бизнес
108,0,108,manager,Всего хорошего до свидания,NaN,NaN,NaN,True,NaN
110,1,1,manager,Алло здравствуйте,True,False,NaN,NaN,NaN
111,1,2,manager,Меня зовут ангелина компания диджитал бизнес з...,NaN,True,ангелина,NaN,диджитал бизнес
163,1,54,manager,До свидания,NaN,NaN,NaN,True,NaN
166,2,2,manager,Алло здравствуйте,True,False,NaN,NaN,NaN
167,2,3,manager,Меня зовут ангелина компания диджитал бизнес з...,NaN,True,ангелина,NaN,диджитал бизнес
250,3,1,manager,Алло дмитрий добрый день,True,False,NaN,NaN,NaN
251,3,2,manager,Добрый меня максим зовут компания китобизнес у...,NaN,True,максим,NaN,китобизнес


**Вывод**

Парсер создан, и на представленном датасете поставленную задачу выполняет.

### Отчёт по решению

#### Задача
Была поставлена задача по созданию скрипта для парсинга диалогов. 
Главные задачи, которые должен был выполнять скрипт:
- Извлекать реплики с приветствием – где менеджер поздоровался. 
- Извлекать реплики, где менеджер представил себя. 
- Извлекать имя менеджера. 
- Извлекать название компании. 
- Извлекать реплики, где менеджер попрощался.
- Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»

#### Анализ данных

Анализ данных показал, что данных для создания парсера, который будет хорошо работать на любых текстах, явно не достаточно. Создаваемый скрипт будет неким бейзлайном для дальнейшей работы. 

Для поиска и извлечения заданных сущностей было решено использовать библиотеку `Natasha` и `Yargy`, так как они позволяют без использования "тяжелых" моделей искать и извлекать необходимые нам данные. 

### Парсер

Работа по созданию скрипта для парсинга была построена следующим образом:
1. Извлечение реплик с приветствием и реплик, где менеджер попрощался - использовал простой парсер из библиотеки `Yargy` на основе поиска слов из заданного списка.
2. Извлечение имени менеджера: с помошью Yargy парсера находились слова указывающие на представление, затем извлекались слова которые могли быть именами с последующей проверкой.
3. Извлечение названий компаний происводилось аналогично извлечению имён менеджеров. Так как названия компаний могли состоять из двух слов, проверялась синтаксическая связь между ними, и если таковая была, принималось решение о том, что второе слово тоже отностися к названию компании.
4. Все парсеры были объеденены в одной функции, поторая была применена к каждой строке набора данных.
5. В результате работы скрипта к датасету были добавлены колонки с результатами поиска. Паралельно заполнялась таблица результатов.
6. Проверку требования к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом», была осуществлена после работы основного парсера, путём обработки итоговой таблицы. 


### Итог работы скрипта

Исходя из оценки работы созданного парсера, все заданные сущности были найдены. Как уже было отмечено, данный скрипт необходимо дорабатывать на большем датасете, и тестировать на данных, которые не попадали в тренировочный датасет. Такой возможности в данном проекте небыло, так как здесь было представлено всего шесть диалогов, которые делить на трейн и тест было не целесообразно.

### Проблемы, возникшие в ходе создания скрипта

1. NameExtractor из библиотеки Natasha может иногда удивить. Определяет некоторые обычные слова как имена, а некоторые имена не определяет. Решить эту проблему удалось извлечением имён не просто принемив метод NameExtractor, а сначала нийти где это имя может быть, а затем уже проверять слова-кандидаты имя это или нет.
2. С извлечением имён компаний ситуация тоже не однозначная. Возможно, проще было-бы собрать словарь из названий компаний, но их в датасете всего две. Поэтому пришлось искать связи междк словами, притендующими на роль названия компании. В других диалога, может и не быть синтаксической связи между словами в названии компании. И слов в имени компании может быть больше двух. Имея больший датасет, надо будет подумать как лучше извлекать названия компаний.


## Вывод

Считаю, что задача по созданию скрипта по парсингу диалогов выполнена. Скрипт находит и извлекает необходимые данные в рамках представленных данных.